In [ ]:
import sys
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr

%matplotlib notebook

In [ ]:
import qcodes as qc

In [ ]:
from importlib import reload
from qctools import instruments as instools
reload(instools)

import labpythonconfig as cfg
reload(cfg)

from experiment import measurement
reload(measurement)
from experiment.measurement import Parameter, BaseMeasurement

In [ ]:
station = qc.Station()

In [ ]:
class DummyMeasurement(BaseMeasurement):
    def __init__(self, station):
        super().__init__(station)
        
        self.add_parameter('start', Parameter)
        self.add_parameter('stop', Parameter)
        self.add_parameter('npts', Parameter)
    
    def measure(self):
        # initialize the data we need
        xpoints = np.linspace(self.start(), self.stop(), self.npts())
        data = xr.DataArray(np.zeros_like(xpoints),
                            coords=[('xpoints', xpoints)])
        self.dataset['data'] = data
        self.save_data()
        
        # now populate in a primitive loop
        for i, x in enumerate(xpoints):
            
            # acquire some data, and save right away
            self.dataset['data'][i] = np.cos(x)
            self.save_data()                       
                    
        

m = DummyMeasurement(station)
m.start(0)
m.stop(1)
m.npts(11)
m.run()

In [ ]:
fn = r"C:\data\2017-12\2017-12-17\2017-12-17_DummyMeasurement-#0003.hdf5"
with xr.open_dataset(fn, 'r') as ds:
    print(ds['data'])
ds.close()

In [ ]:
import h5py
f = h5py.File(fn, 'r')

In [ ]:
x = np.linspace(0,10,101)
y = np.cos(x)

In [ ]:
da = xr.DataArray(y, coords=[('x', x)])
ds = xr.Dataset({'y' : da})
ds.to_netcdf('test.h5', format='NETCDF4')

In [ ]:
ds

In [ ]:
np.exp(-0.5/50.)

In [ ]:
0.3 * 140**.5

In [ ]:
0.5 * 0.3e-12 * 